The aim of this notebook is to create a decent vanilla Convolutional Neural Network that is capable of distinguishing between a Dog and Cat.

Please note that it is advisable to use pre-trained models for image classification tasks, but i'll be creating my own model to solidify my understanding of DNNs and CNNs.


## Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import random

import os
print(os.listdir("../input/dogs-vs-cats"))

In [ ]:
# Unzipping the files

import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")  

In [ ]:
# Retrieving a list of directories in each folder

DIR_TRAIN = "/kaggle/working/train/"
DIR_TEST = "/kaggle/working/test1"

train_imgs = os.listdir(DIR_TRAIN)
test_imgs = os.listdir(DIR_TEST)

In [ ]:
train_imgs[:5]

## Loading a sample image

In [ ]:
sample = random.choice(train_imgs)
image = load_img("/kaggle/working/train/"+sample)
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
# Creating a DataFrame for our train set

category = [x.split(".")[0] for x in train_imgs]
df = pd.DataFrame({"Filename":train_imgs, "Category":category})
df.head()

In [ ]:
# Visualizing the constituents of our train set

plt.figure(figsize=(6,6))
plt.pie(df['Category'].value_counts(), explode=[0.01,0.02], 
       autopct="%.2f%%", textprops={'color':'white', 'size':12,
                                   'weight':'bold'},
       startangle=45, colors = ['#947867', '#D49034'])
plt.legend(["Dogs","Cats"])
plt.show()

### Splitting the data into train & validation

In [ ]:
# Splitting the train set, into a train & validation set with equal categories

df_train, df_valid = train_test_split(df, test_size = 5000, 
                                     stratify=df['Category'],
                                     random_state=42)

In [ ]:
# Checking if categories are equal

df_train['Category'].value_counts()

In [ ]:
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

## 2.0 PROCESSING THE DATA

### Data Augmentation

In [ ]:
# Creating an Augmentation generator for the train set

train_datagen = ImageDataGenerator(
    rotation_range = 15,
    rescale = 1.0/255.0,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
)

train_generator = train_datagen.flow_from_dataframe(df_train, 
                                  directory = "/kaggle/working/train/",
                                  x_col = 'Filename',
                                  y_col = 'Category',
                                  target_size = (224, 224),
                                  class_mode = 'categorical',
                                  batch_size = 32
                                 )

In [ ]:
# Creating the Augmentation generator for the valid set

valid_datagen = ImageDataGenerator(rescale = 1.0/255.0)

valid_generator = valid_datagen.flow_from_dataframe(df_valid, 
                                  directory = "/kaggle/working/train/",
                                  x_col = 'Filename',
                                  y_col = 'Category',
                                  target_size = (224, 224),
                                  class_mode = 'categorical',
                                  batch_size = 32
                                 )

### Let's see how well our Generator works

In [ ]:
df_example = df_train.sample(1)

example_generator = train_datagen.flow_from_dataframe(df_example,
                                  directory = "/kaggle/working/train/",
                                  x_col = 'Filename',
                                  y_col = 'Category',
                                  target_size = (224, 224),
                                  class_mode = 'categorical')

In [ ]:
plt.figure(figsize=(12,12))

for i in range(9):
    plt.subplot(3, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        plt.axis("off")
        break
plt.tight_layout()
plt.show()

Works as we had hoped !!

# 3.0 CREATING THE MODEL

### Creating the sequential vanilla model

In [ ]:
from functools import partial

keras.backend.clear_session()


DefaultConv = partial(keras.layers.Conv2D, kernel_size = 3, strides = 1,
                     padding = 'same', activation = 'relu')

model = keras.models.Sequential([
    DefaultConv(filters = 32, kernel_size = 7, strides=2, input_shape=[224,224,3]),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),

    DefaultConv(filters = 64),
    DefaultConv(filters = 64),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.MaxPool2D(pool_size=2),
    
    DefaultConv(filters = 128),
    DefaultConv(filters = 128),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.MaxPool2D(pool_size=2),
    
    DefaultConv(filters = 256),
    DefaultConv(filters = 256),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.MaxPool2D(pool_size=2),
    
    DefaultConv(filters = 512),
    DefaultConv(filters = 512),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation = 'relu', use_bias = False),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation = 'sigmoid')
])

model.compile(loss = "binary_crossentropy", optimizer = 'nadam',
             metrics = ['accuracy'])

In [ ]:
model.summary()

### callbacks

In [ ]:
Checkpoint_cb = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
Earlystopping_cb = keras.callbacks.EarlyStopping(patience=10)

callback = [Checkpoint_cb, Earlystopping_cb]

### Fitting the model

In [ ]:
history = model.fit(
    train_generator,
    epochs = 50,
    validation_data = valid_generator,
    validation_steps = len(df_valid)/32,
    steps_per_epoch = len(df_train)/32,
    callbacks=callback
)

## Visualizing the Training

In [ ]:
# Train and Validation Loss

plt.figure(figsize=(12,6))
plt.plot(history.history['loss'][1:], "ro-", label = "Train Loss")
plt.plot(history.history['val_loss'][1:], "b--", lw=3, label = "Validation Loss")
plt.legend(loc="upper right", fontsize=12)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.title("Train & Validation Loss (50 epochs)", fontsize=16)
plt.show()

In [ ]:
# Train and Validation Accuracy

plt.figure(figsize=(12,6))
plt.plot(history.history['accuracy'], "ro-", label = "Train Accuracy")
plt.plot(history.history['val_accuracy'], "b--", lw=3, label = "Validation Accuracy")
plt.legend(loc="lower right", fontsize=12)
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.title("Train & Validation Accuracy (50 epochs)", fontsize=16)
plt.show()

We can see from our chart that during training, we experienced some cases of overfitting. In general, the model performed well due to the addition of regularization and sensitivity techniques like `DataAugmentaion`, `BatchNormalization` and `DropOut`.

## PROCESSING THE TEST DATA

In [ ]:
df_test = pd.DataFrame({'Filename':test_imgs})
df_test.head()

## Creating the Generator

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    directory = "/kaggle/working/test1/",
    x_col = 'Filename',
    y_col = None,
    class_mode = None,
    target_size = (224, 224),
    batch_size = 32,
    shuffle = False
)

In [ ]:
# loading the best model

model = keras.models.load_model("model.h5")

In [ ]:
model.summary()

## Prediction

In [ ]:
pred = model.predict(test_generator, 
                     steps = np.ceil(df_test.shape[0]/32))

The model returns the probability an instace belongs to each category.

In [ ]:
np.set_printoptions(suppress=True)

pred[:5]

We will add a category to our dataframe based on our predictions. If the first probabilty on each row is greater than 0.5, we assign it as `cat`, else `dog`.

In [ ]:
category = []
for x in pred[:,0]:
    category.append("cat" if x > 0.5 else "dog")
    
df_test['Category'] = category

df_test.head()

## Visualizing our predictions

In [ ]:
plt.figure(figsize=(6,6))
plt.pie(df_test['Category'].value_counts(), explode=[0.01,0.02], 
       autopct="%.2f%%", textprops={'color':'white', 'size':12,
                                   'weight':'bold'},
       startangle=45, colors = ['#947867', '#D49034'])
plt.legend(["Dogs","Cats"])
plt.show()

In [ ]:
df_example = df_test.sample(50).reset_index(drop=True)

plt.figure(figsize=(18,12))

for i in range(50):
    plt.subplot(5,10,i+1)
    filename = df_example['Filename'][i]
    category = df_example['Category'][i]
    image = load_img('/kaggle/working/test1/'+filename)
    plt.imshow(image)
    plt.title(f"Prediction: {category}")
    plt.axis("off")
plt.tight_layout()
plt.show()

For the submission, we are to provide the probabilty that an image is a dog. Therefore, i will be extracting the second column of our `pred` variable for this purpose.

In [ ]:
df_submission = df_test.copy()
df_submission['Category'] = pred[:,1]
df_submission['id'] = df_submission['Filename'].str.split('.').str[0]
df_submission['label'] = df_submission['Category']
df_submission.drop(['Filename', 'Category'], axis = 1, inplace = True)
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_submission

# $The$ $End!$